<a href="https://colab.research.google.com/github/InDataSight/GrowthPan/blob/main/ETLconceptworkflow1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.0 Setup
### 1.1 Folder structure
### 1.2 Install libraries and modules
### 1.3 Test

In [28]:
# SETUP
text

NameError: name 'text' is not defined

In [4]:
#!mkdir ScrapyProfesiaRawData ScrapyProfesiaProcessedData ScrapyProfesiaLogs

mkdir: cannot create directory ‘ScrapyProfesiaRawData’: File exists
mkdir: cannot create directory ‘ScrapyProfesiaProcessedData’: File exists
mkdir: cannot create directory ‘ScrapyProfesiaLogs’: File exists


In [23]:
import subprocess

#Run pip for defined modules in the requirements.txt
pip_install_result = subprocess.run(['pip', 'install', '-r', 'requirements.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

#output results for the pip_install_result
if pip_install_result.returncode == 0:
    print("Pip install successful")
else:
    print("Pip install failed")


Pip install successful


In [16]:
!pytest ScrapyProfesiaSetupTest.py

======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0
rootdir: /content
plugins: typeguard-4.4.1, anyio-3.7.1
collected 1 item                                                                                   

ScrapyProfesiaSetupTest.py .                                                                 [100%]

======================================== 1 passed in 0.01s =========================================


NameError: name 'python' is not defined